In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import timm
import time

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# === CONFIG === #
BATCH_SIZE = 8
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_DIR = "/content/drive/MyDrive/datatset/train"
TEST_DIR = "/content/drive/MyDrive/datatset/test"

# === TRANSFORMS === #
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


# === DATASET === #
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform

        for label_name in ["healthy", "sick"]:
            class_root = os.path.join(root_dir, label_name)
            if not os.path.exists(class_root):
                continue


            for folder in os.listdir(class_root):
                sub_path = os.path.join(class_root, folder, "Segmentadas")
                if not os.path.exists(sub_path):
                    continue

                for file_name in os.listdir(sub_path):
                    if file_name.lower().endswith((".jpg", ".jpeg", ".png")):
                        full_path = os.path.join(sub_path, file_name)

                        self.images.append(full_path)
                        self.labels.append(0 if label_name == "healthy" else 1)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label



In [ ]:
!ls /content/drive/MyDrive/datatset/

test  train


In [ ]:
# === LOADERS === #
train_dataset = CustomDataset(TRAIN_DIR, transform=transform)
test_dataset = CustomDataset(TEST_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# === MODEL WRAPPER === #
def get_model(model_name):
    model = timm.create_model(model_name, pretrained=True, num_classes=2)
    return model.to(DEVICE)



In [ ]:
# # === TRAINING === #
# def train_model(model):
#     model.train()
#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

#     for epoch in range(NUM_EPOCHS):
#         for images, labels in train_loader:
#             images, labels = images.to(DEVICE), labels.to(DEVICE)
#             outputs = model(images)
#             loss = criterion(outputs, labels)

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

# # === EVALUATION === #
# def evaluate_model(model):
#     model.eval()
#     y_true, y_pred = [], []
#     start_time = time.time()

#     with torch.no_grad():
#         for images, labels in test_loader:
#             images = images.to(DEVICE)
#             outputs = model(images)
#             preds = torch.argmax(outputs, dim=1).cpu().numpy()
#             y_true.extend(labels.numpy())
#             y_pred.extend(preds)

#     end_time = time.time()
#     cm = confusion_matrix(y_true, y_pred)
#     tn, fp, fn, tp = cm.ravel()
#     acc = accuracy_score(y_true, y_pred)
#     sens = tp / (tp + fn) if (tp + fn) > 0 else 0
#     speed = end_time - start_time
#     return acc, sens, speed



# # === ENSEMBLE MODEL COMBINING SWIN + CONVNEXT + MOBILEVIT === #
# class EnsembleModel(nn.Module):
#     def __init__(self):
#         super(EnsembleModel, self).__init__()
#         self.swin = get_model("swin_tiny_patch4_window7_224")
#         self.convnext = get_model("convnext_tiny.fb_in22k")
#         self.mobilevit = get_model("mobilevit_s")

#         self.swin.fc = nn.Identity()
#         self.convnext.head = nn.Identity()
#         self.mobilevit.classifier = nn.Identity()

#         self.classifier = nn.Sequential(
#             nn.Linear(3 * 1000, 512),
#             nn.ReLU(),
#             nn.Linear(512, 2)
#         )

#     def forward(self, x):
#         swin_feat = self.swin(x)
#         convnext_feat = self.convnext(x)
#         mobilevit_feat = self.mobilevit(x)

#     # Handle 4D to 2D flattening if needed
#         if swin_feat.ndim == 4:
#             swin_feat = self.pool(swin_feat).flatten(1)
#         if convnext_feat.ndim == 4:
#             convnext_feat = self.pool(convnext_feat).flatten(1)
#         if mobilevit_feat.ndim == 4:
#             mobilevit_feat = self.pool(mobilevit_feat).flatten(1)

#         combined = torch.cat((swin_feat, convnext_feat, mobilevit_feat), dim=1)
#         return self.classifier(combined)


# # === MAIN === #
# print("\n🌀 Training Ensemble Model (Swin + ConvNeXt + MobileViT)...")
# ensemble_model = EnsembleModel()
# train_model(ensemble_model)
# acc, sens, speed = evaluate_model(ensemble_model)

# print(f"\n🎯 Ensemble Model → Accuracy: {acc:.4f}, Sensitivity: {sens:.4f}, Inference Time: {speed:.2f}s")


In [ ]:
# === TRAINING === #
def train_model(model):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

# === EVALUATION === #
from sklearn.metrics import accuracy_score, confusion_matrix

def evaluate_model(model):
    model.eval()
    y_true, y_pred = [], []
    start_time = time.time()

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    end_time = time.time()
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    acc = accuracy_score(y_true, y_pred)
    sens = tp / (tp + fn) if (tp + fn) > 0 else 0
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0
    speed = end_time - start_time

    return acc, sens, prec, spec, speed


# === ENSEMBLE MODEL COMBINING SWIN + CONVNEXT + MOBILEVIT === #
class EnsembleModel(nn.Module):
    # def __init__(self):
    #     super(EnsembleModel, self).__init__()
    #     self.swin = get_model("swin_tiny_patch4_window7_224")
    #     self.convnext = get_model("convnext_tiny.fb_in22k")
    #     self.mobilevit = get_model("mobilevit_s")

    #     self.swin.fc = nn.Identity()
    #     self.convnext.head = nn.Identity()
    #     self.mobilevit.classifier = nn.Identity()

    #     self.pool = nn.AdaptiveAvgPool2d((1, 1))

    #     self.classifier = nn.Sequential(
    #         nn.Linear(input_dim, 512),
    #         nn.ReLU(),
    #         nn.Linear(512, 2)
    #     )
    def __init__(self):
       super(EnsembleModel, self).__init__()
       self.swin = get_model("swin_tiny_patch4_window7_224")
       self.convnext = get_model("convnext_tiny.fb_in22k")
       self.mobilevit = get_model("mobilevit_s")

       self.swin.fc = nn.Identity()
       self.convnext.head = nn.Identity()
       self.mobilevit.classifier = nn.Identity()

       self.pool = nn.AdaptiveAvgPool2d((1, 1))

    # === Dynamically infer feature size === #
       with torch.no_grad():
           dummy = torch.randn(1, 3, 224, 224).to(DEVICE)
           swin_feat = self.swin(dummy)
           conv_feat = self.convnext(dummy)
           mobi_feat = self.mobilevit(dummy)

           if swin_feat.ndim == 4:
               swin_feat = self.pool(swin_feat).flatten(1)
           if conv_feat.ndim == 4:
               conv_feat = self.pool(conv_feat).flatten(1)
           if mobi_feat.ndim == 4:
               mobi_feat = self.pool(mobi_feat).flatten(1)

           input_dim = swin_feat.size(1) + conv_feat.size(1) + mobi_feat.size(1)

    # === Classifier using correct input dim === #
       self.classifier = nn.Sequential(
           nn.Linear(input_dim, 512),
           nn.ReLU(),
           nn.Linear(512, 2)
       )


    def forward(self, x):
        swin_feat = self.swin(x)
        convnext_feat = self.convnext(x)
        mobilevit_feat = self.mobilevit(x)

        if swin_feat.ndim == 4:
            swin_feat = self.pool(swin_feat).flatten(1)
        if convnext_feat.ndim == 4:
            convnext_feat = self.pool(convnext_feat).flatten(1)
        if mobilevit_feat.ndim == 4:
            mobilevit_feat = self.pool(mobilevit_feat).flatten(1)

        combined = torch.cat((swin_feat, convnext_feat, mobilevit_feat), dim=1)
        return self.classifier(combined)

# === MAIN WORKFLOW === #
def run_and_log(name):
    model = get_model(name)
    print(f"\n🧪 Training {name}...")
    train_model(model)
    acc, sens, prec, spec, speed = evaluate_model(model)
    print(f"✅ {name} → Acc: {acc:.4f}, Sens: {sens:.4f}, Prec: {prec:.4f}, Spec: {spec:.4f}, Time: {speed:.2f}s")

    return acc, sens, prec, spec, speed


results = {}

for model_name in ["swin_tiny_patch4_window7_224", "convnext_tiny.fb_in22k", "mobilevit_s"]:
    acc, sens, prec, spec, speed = run_and_log(model_name)
    results[model_name] = {
        "accuracy": acc,
        "sensitivity": sens,
        "precision": prec,
        "specificity": spec,
        "time": speed
    }




🧪 Training swin_tiny_patch4_window7_224...
✅ swin_tiny_patch4_window7_224 → Acc: 0.6406, Sens: 0.7000, Prec: 0.6257, Spec: 0.5813, Time: 4.08s


model.safetensors:   0%|          | 0.00/178M [00:00<?, ?B/s]


🧪 Training convnext_tiny.fb_in22k...
✅ convnext_tiny.fb_in22k → Acc: 0.6188, Sens: 0.7375, Prec: 0.5960, Spec: 0.5000, Time: 3.58s


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]


🧪 Training mobilevit_s...
✅ mobilevit_s → Acc: 0.6813, Sens: 0.7750, Prec: 0.6526, Spec: 0.5875, Time: 3.25s


In [ ]:
print("\n🌀 Training Ensemble Model (Swin + ConvNeXt + MobileViT)...")
ensemble_model = EnsembleModel().to(DEVICE)
train_model(ensemble_model)
acc, sens, prec, spec,speed = evaluate_model(ensemble_model)
results["ensemble"] = {"accuracy": acc, "sensitivity": sens, "precision": prec,"specificity": spec, "time": speed}

print(f"\n✅ Ensemble → Accuracy: {acc:.4f}, Sensitivity: {sens:.4f}, Precision: {prec:.4f}, Specificity: {spec:.4f}, Time: {speed:.2f}s")

# Save ensemble model
torch.save(ensemble_model.state_dict(), "ensemble_model.pth")
print("\n💾 Ensemble model saved as 'ensemble_model.pth'")

# Print summary
print("\n📊 Final Evaluation Summary:")
for model, metrics in results.items():
    print(f"{model}: Accuracy={metrics['accuracy']:.4f}, Sensitivity={metrics['sensitivity']:.4f}, Time={metrics['time']:.2f}s")



🌀 Training Ensemble Model (Swin + ConvNeXt + MobileViT)...

✅ Ensemble → Accuracy: 0.8000, Sensitivity: 0.8625, Precision: 0.7667, Specificity: 0.7375, Time: 5.68s

💾 Ensemble model saved as 'ensemble_model.pth'

📊 Final Evaluation Summary:
swin_tiny_patch4_window7_224: Accuracy=0.6406, Sensitivity=0.7000, Time=4.08s
convnext_tiny.fb_in22k: Accuracy=0.6188, Sensitivity=0.7375, Time=3.58s
mobilevit_s: Accuracy=0.6813, Sensitivity=0.7750, Time=3.25s
ensemble: Accuracy=0.8000, Sensitivity=0.8625, Time=5.68s
